In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
# PINN model
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 3)
        )

    def forward(self, x, y, t):
        input_data = torch.stack([x.view(-1), y.view(-1), t.view(-1)], dim=1)
        return self.model(input_data)


def initial_conditions(x,y):
    u_initial = torch.zeros_like(x)
    v_initial = torch.zeros_like(y)
    return u_initial, v_initial

# boundary condition (viscous wall)
def boundary_conditions_wall(x,y):
    u_boundary = torch.zeros_like(x) 
    v_boundary = torch.zeros_like(y)  
    return u_boundary, v_boundary

# boundary condition (inflow)
def boundary_conditions_inflow(x,y,t):
    U = 1.5#*torch.sin(np.pi*t/8)
    u_boundary = 4*U*(y)*(0.41-y)/(0.41*0.41)
    v_boundary = torch.zeros_like(y)  
    return u_boundary, v_boundary

# boundary condition (outflow)
def boundary_conditions_outflow(x,y):
    p_boundary = torch.zeros_like(x)  
    return p_boundary

# NS equation (for steady)
def navier_stokes_residual(uvp,x,y,t):
    u, v, p = torch.chunk(uvp, 3, dim=1) 

    # Constants
    rho = 1  # Density
    mu = 0.001  # Viscosity coefficient
    
    # Compute derivatives using automatic differentiation
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]

    v_t = torch.autograd.grad(v.sum(), t, create_graph=True)[0]
    v_x = torch.autograd.grad(v.sum(), x, create_graph=True)[0]
    v_y = torch.autograd.grad(v.sum(), y, create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x.sum(), x, create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y.sum(), y, create_graph=True)[0]

    p_x = torch.autograd.grad(p.sum(), x, create_graph=True)[0]
    p_y = torch.autograd.grad(p.sum(), y, create_graph=True)[0] 

    # Navier-Stokes equations for u and v momentum
    residual_u = rho*(u_t + u * u_x + v * u_y) - mu*(u_xx + u_yy) + p_x
    residual_v = rho*(v_t + u * v_x + v * v_y) - mu*(v_xx + v_yy) + p_y
    
    # Continuity equation for incompressibility condition
    residual_p = (u_x + v_y)

    return residual_u, residual_v, residual_p

In [ ]:
# initial condition
x_initial = np.linspace(0, 2.2, 440).reshape(-1, 1)
y_initial = np.linspace(0, 0.41, 82).reshape(-1, 1)
x_initial, y_initial = np.meshgrid(x_initial, y_initial)
cylinder_mask = (x_initial - 0.2) ** 2 + (y_initial - 0.2) ** 2 > 0.05 ** 2
x_initial = torch.tensor(x_initial[cylinder_mask], dtype=torch.float64, device="cuda", requires_grad=True)
y_initial = torch.tensor(y_initial[cylinder_mask], dtype=torch.float64, device="cuda", requires_grad=True)


# time
t = np.linspace(0, 10, 51)

# wall boundary (Cylinder)
cylinder_points = 100
theta = np.linspace(0, 2 * np.pi, cylinder_points).reshape(-1, 1)
x_wall_boundary = torch.tensor(0.2 + np.cos(theta) * 0.05)
y_wall_boundary = torch.tensor(0.2 + np.sin(theta) * 0.05)
cylinder_wall_data = torch.tensor([[x_wall_boundary[i], y_wall_boundary[i], t[j]] for i in range(len(x_wall_boundary)) for j in range(len(t))], dtype=torch.float64)
# cylinder_wall_data = cylinder_wall_data[torch.randperm(len(cylinder_wall_data))].cuda()
cylinder_wall_data = cylinder_wall_data.cuda()
cylinder_wall_data.requires_grad = True


# inflow boundary
y_inflow_boundary = torch.linspace(0, 0.41, 100).reshape(-1, 1)
x_inflow_boundary = torch.zeros_like(y_inflow_boundary)
inflow_data = torch.tensor([[x_inflow_boundary[i], y_inflow_boundary[i], t[j]] for i in range(len(x_inflow_boundary)) for j in range(len(t))], dtype=torch.float64)
# inflow_data = inflow_data[torch.randperm(len(inflow_data))].cuda()
inflow_data = inflow_data.cuda()
inflow_data.requires_grad = True


# outflow boundary
y_outflow_boundary = torch.linspace(0, 0.41, 100).reshape(-1, 1)
x_outflow_boundary = torch.full_like(y_outflow_boundary, 2.2)
outflow_data = torch.tensor([[x_outflow_boundary[i], y_outflow_boundary[i], t[j]] for i in range(len(x_outflow_boundary)) for j in range(len(t))], dtype=torch.float64)
# outflow_data = outflow_data[torch.randperm(len(outflow_data))].cuda()
outflow_data = outflow_data.cuda()
outflow_data.requires_grad = True

# wall boundary (up)
x_up_wall_boundary = torch.linspace(0, 2.2, 100).reshape(-1, 1)
y_up_wall_boundary = torch.full_like(x_up_wall_boundary, 0.41)
up_wall_data = torch.tensor([[x_up_wall_boundary[i], y_up_wall_boundary[i], t[j]] for i in range(len(x_up_wall_boundary)) for j in range(len(t))], dtype=torch.float64)
# up_wall_data = up_wall_data[torch.randperm(len(up_wall_data))].cuda()
up_wall_data = up_wall_data.cuda()
up_wall_data.requires_grad = True

# wall boundary (down)
x_down_wall_boundary = torch.linspace(0, 2.2, 100).reshape(-1, 1)
y_down_wall_boundary = torch.full_like(x_down_wall_boundary, 0)
down_wall_data = torch.tensor([[x_down_wall_boundary[i], y_down_wall_boundary[i], t[j]] for i in range(len(x_down_wall_boundary)) for j in range(len(t))], dtype=torch.float64)
# down_wall_data = down_wall_data[torch.randperm(len(down_wall_data))].cuda()
down_wall_data = down_wall_data.cuda()
down_wall_data.requires_grad = True



# wall boundary (Cylinder)
cylinder_points = 20
theta = np.linspace(0, 2 * np.pi, cylinder_points).reshape(-1, 1)
radius_list = [0.06, 0.07, 0.08, 0.09, 0.1]
x_cylinder_collocation = []
y_cylinder_collocation = []
t_cylinder_collocation = []


for t_ in t:
   for r_ in radius_list:
      x = 0.2 + np.cos(theta)*r_
      y = 0.2 + np.sin(theta)*r_

      for i in range(len(theta)):
         x_cylinder_collocation.append(x[i])
         y_cylinder_collocation.append(y[i])
         t_cylinder_collocation.append(t_)

x_cylinder_collocation = torch.tensor(np.array(x_cylinder_collocation).reshape(-1, 1), dtype=torch.float64, device="cuda", requires_grad=True)
y_cylinder_collocation = torch.tensor(np.array(y_cylinder_collocation).reshape(-1, 1), dtype=torch.float64, device="cuda", requires_grad=True)
t_cylinder_collocation = torch.tensor(np.array(t_cylinder_collocation).reshape(-1, 1), dtype=torch.float64, device="cuda", requires_grad=True)


# NS collocation points
x_collocation = []
y_collocation = []
t_collocation = []
cnt = 0
while True:

   y_c = np.random.uniform(0, 0.41)
   x_c = np.random.uniform(0, 2.2)   
   t_c = np.random.uniform(0, 10)
   
   if not ((x_c - 0.2) ** 2 + (y_c - 0.2) ** 2 < 0.05 ** 2):
      x_collocation.append(x_c)
      y_collocation.append(y_c)
      t_collocation.append(t_c)
      cnt += 1
      if cnt > 40000:
         break

x_collocation = torch.tensor(np.array(x_collocation).reshape(-1, 1), dtype=torch.float64, device="cuda", requires_grad=True)
y_collocation = torch.tensor(np.array(y_collocation).reshape(-1, 1), dtype=torch.float64, device="cuda", requires_grad=True)
t_collocation = torch.tensor(np.array(t_collocation).reshape(-1, 1), dtype=torch.float64, device="cuda", requires_grad=True)


In [ ]:
def train_PINN(model,x_initial, y_initial, cylinder_wall_data, up_wall_data, down_wall_data, inflow_data, outflow_data, x_collocation, y_collocation, t_collocation, x_cylinder_collocation, y_cylinder_collocation, t_cylinder_collocation , epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # optimizer = optim.LBFGS(model.parameters(), lr = learning_rate, history_size=10, max_iter=10,  line_search_fn="strong_wolfe")
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min= 0.00001,T_max=1000)

    loss_fn = nn.MSELoss()
    
    def closure():
        optimizer.zero_grad()

        solutions_initial_pred = model(x_initial, y_initial, torch.zeros_like(x_initial))
        solutions_initial_exact = initial_conditions(x_initial, y_initial)
        loss1 = loss_fn(solutions_initial_pred[:,0].view(-1, 1), solutions_initial_exact[0].view(-1,1))
        loss2 = loss_fn(solutions_initial_pred[:,1].view(-1, 1), solutions_initial_exact[1].view(-1,1))
        
        solutions_collocation_pred = model(x_collocation, y_collocation, t_collocation)    
        equation_residual_u, equation_residual_v, equation_residual_p = navier_stokes_residual(solutions_collocation_pred, x_collocation, y_collocation, t_collocation)
        loss3 = loss_fn(equation_residual_u, torch.zeros_like(equation_residual_u))
        loss4 = loss_fn(equation_residual_v, torch.zeros_like(equation_residual_v))
        loss5 = loss_fn(equation_residual_p, torch.zeros_like(equation_residual_p))
        # print(loss)

        solutions_wall_boundary_pred = model(cylinder_wall_data[:,0], cylinder_wall_data[:,1], cylinder_wall_data[:,2])
        solutions_wall_boundary_exact = boundary_conditions_wall(cylinder_wall_data[:,0], cylinder_wall_data[:,1])
        loss6 = loss_fn(solutions_wall_boundary_pred[:,0].view(-1, 1), solutions_wall_boundary_exact[0].view(-1,1))
        loss7 = loss_fn(solutions_wall_boundary_pred[:,1].view(-1, 1), solutions_wall_boundary_exact[1].view(-1,1))
        # print(loss)

        solutions_up_wall_boundary_pred = model(up_wall_data[:,0], up_wall_data[:,1], up_wall_data[:,2])
        solutions_up_wall_boundary_exact = boundary_conditions_wall(up_wall_data[:,0], up_wall_data[:,1])
        loss8 = loss_fn(solutions_up_wall_boundary_pred[:,0].view(-1, 1), solutions_up_wall_boundary_exact[0].view(-1,1))
        loss9 = loss_fn(solutions_up_wall_boundary_pred[:,1].view(-1, 1), solutions_up_wall_boundary_exact[1].view(-1,1))
        # print(loss)

        solutions_down_wall_boundary_pred = model(down_wall_data[:,0], down_wall_data[:,1],down_wall_data[:,2])
        solutions_down_wall_boundary_exact = boundary_conditions_wall(down_wall_data[:,0], down_wall_data[:,1])
        loss10 = loss_fn(solutions_down_wall_boundary_pred[:,0].view(-1, 1), solutions_down_wall_boundary_exact[0].view(-1,1))
        loss11 = loss_fn(solutions_down_wall_boundary_pred[:,1].view(-1, 1), solutions_down_wall_boundary_exact[1].view(-1,1))
        # print(loss)

        solutions_inflow_boundary_pred = model(inflow_data[:,0], inflow_data[:,1],inflow_data[:,2])
        solutions_inflow_boundary_exact = boundary_conditions_inflow(inflow_data[:,0], inflow_data[:,1], inflow_data[:,2])
        loss12 = loss_fn(solutions_inflow_boundary_pred[:,0].view(-1, 1), solutions_inflow_boundary_exact[0].view(-1,1))
        loss13 = loss_fn(solutions_inflow_boundary_pred[:,1].view(-1, 1), solutions_inflow_boundary_exact[1].view(-1,1))
        # print(loss)

        solutions_outflow_boundary_pred = model(outflow_data[:,0], outflow_data[:,1],outflow_data[:,2])
        solutions_outflow_boundary_exact = boundary_conditions_outflow(outflow_data[:,0], outflow_data[:,1])
        loss14 = loss_fn(solutions_outflow_boundary_pred[:,2].view(-1, 1), solutions_outflow_boundary_exact.view(-1,1))
        # print(loss)

        solutions_collocation2_pred = model(x_cylinder_collocation, y_cylinder_collocation, t_cylinder_collocation)
        equation_residual2_u, equation_residual2_v, equation_residual2_p = navier_stokes_residual(solutions_collocation2_pred, x_cylinder_collocation, y_cylinder_collocation, t_cylinder_collocation)
        loss15 = loss_fn(equation_residual2_u, torch.zeros_like(equation_residual2_u))
        loss16 = loss_fn(equation_residual2_v, torch.zeros_like(equation_residual2_v))
        loss17 = loss_fn(equation_residual2_p, torch.zeros_like(equation_residual2_p))

        print("ic", loss1.item(), loss2.item())
        print("collocation", loss3.item(), loss4.item(), loss5.item())
        print("collocation2", loss15.item(), loss16.item(), loss17.item())
        print("cylinder wall", loss6.item(), loss7.item())
        print("up wall", loss8.item(), loss9.item())
        print("down wall", loss10.item(), loss11.item())
        print("inflow", loss12.item(), loss13.item())
        print("outflow", loss14.item())
        print()
        
        loss = loss1+loss2+loss3+loss4+loss5+loss6+loss7+loss8+loss9+loss10+loss11+loss12+loss13+loss14+loss15+loss16+loss17
        loss.backward()
        return loss
        
    min_loss = 10
    for epoch in range(epochs):
        loss_ = optimizer.step(closure)
        #scheduler.step()
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss_.item()}")
        
        if (torch.isnan(loss_)):
            break

        if (epoch%10==0):
            torch.save(model.state_dict(), "model/test_current.pt")

        if (loss_.item()<min_loss):
            torch.save(model.state_dict(), "model/test_current_min1.pt")
            min_loss = loss_.item()
        

    print("Training completed.")


In [ ]:
# 모델 생성 및 학습 
model = PINN().double().cuda()
# model.load_state_dict(torch.load("model/test_current_min5.pt"))

In [6]:
train_PINN(model, x_initial, y_initial, cylinder_wall_data, up_wall_data, down_wall_data, inflow_data, outflow_data, x_collocation, y_collocation, t_collocation, x_cylinder_collocation, y_cylinder_collocation, t_cylinder_collocation, epochs=50000, learning_rate=0.01)

Epoch [502/50000], Loss: 0.9292092835185143
ic 0.029977146477593714 0.0017057867472396768
collocation 0.000298883446484226 0.0004837261666779747 0.007729104400093195
collocation2 0.0003883856966081663 0.0012434054675681215 0.01828518148884323
cylinder wall 0.13741802339635611 0.017310258610063828
up wall 0.08005742334785515 0.00862688011638615
down wall 0.05953737224026676 0.005491163427952769
inflow 0.5408654943634684 0.0226732063267403
outflow 0.00041240783504124535

Epoch [503/50000], Loss: 0.932503849555239
ic 0.006972452292531172 0.0015136795180845813
collocation 0.0004258369092631696 0.0014673452847847421 0.010309298180662233
collocation2 0.0006306557908303845 0.002106045880939092 0.029998046283136088
cylinder wall 0.03580647613488728 0.011421125582250688
up wall 0.01653632499127005 0.005145059080560181
down wall 0.0074814497940952655 0.002421500384361618
inflow 0.7634295653310339 0.017728316617666884
outflow 0.010682221191546444

Epoch [504/50000], Loss: 0.9240753992479038
ic 0.